In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import matplotlib.pyplot as plt
from IPython.display import Image 
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

import team36
from team36.mnist.vgg import VGG
from team36.attacks.fast_gradient_attack_data_set import FastSignGradientAttackDataSet
from team36.training import validate, accuracy, predict

DIR = '.'
DATA_DIR = f'{DIR}/data'

In [13]:
model = VGG()
state_dict = torch.load(f"{DIR}/checkpoints/mnist-vgg.pth", map_location=torch.device('cpu'))
model.load_state_dict(state_dict)

criterion = nn.CrossEntropyLoss()

In [14]:
test_set = torchvision.datasets.MNIST(root=DATA_DIR, train=False, download=True, 
                                      transform=transforms.ToTensor())

test_loader = torch.utils.data.DataLoader(test_set, batch_size=100, shuffle=False, num_workers=2)

test_accuracy, _, test_loss = validate(None, test_loader, model, criterion)

print(f"Regular Test Accuracy is {test_accuracy}")
print(f"Regular Test Loss is {test_loss}")

/usr/local/anaconda3/envs/cs7643-final-project-cpu/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1623459065530/work/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
/usr/local/anaconda3/envs/cs7643-final-project-cpu/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Trigge

Regular Test Accuracy is 0.9839000105857849
Regular Test Loss is 0.0634659081697464


In [17]:
attack_test_set = torchvision.datasets.MNIST(root=DATA_DIR, train=False, download=True, 
                                             transform=transforms.ToTensor())
attack_test_set = FastSignGradientAttackDataSet(attack_test_set, model, criterion, 
                                                               epsilon=0.25)

attack_test_loader = torch.utils.data.DataLoader(attack_test_set, batch_size=100, 
                                                 shuffle=False, num_workers=0)

attack_test_accuracy, _, attack_test_loss = validate(None, attack_test_loader, model, criterion)
print(f"Attack Test Accuracy is {attack_test_accuracy}")
print(f"Attack Test Loss is {attack_test_loss}")

Attack Test Accuracy is 0.5267999768257141
Attack Test Loss is 1.452913522720337


In [18]:
index = 10

input, _ = test_set[index]
display(transforms.functional.to_pil_image(input))
print(predict(model, input))

input, _ = attack_test_set[index]
display(transforms.functional.to_pil_image(input))
print(predict(model, input))

0


2
